# Building a Recommendation System with Hugging Face Transformers  

Learn how to build the recommendation system with advanced technology.

In [1]:
pip install transformers torch pandas scikit-learn



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# [Colab only] Connect to your Google Drive

In [2]:
import pandas as pd

df = pd.read_csv('anime.csv')

df = df.dropna()
df['description'] = df['name'] +' '+ df['genre'] + ' ' +df['type']+' episodes: '+ df['episodes']

In the code above, we read the dataset with Pandas and dropped all the missing data. 

Then, we create a feature called "description" that contains all the information from the available data, such as 
1. name
2. genre
3. type
4. episode number.

The new column would become our basis for the recommendation system. It would be better to have more complete information, such as the anime plot and summary, but let’s be content with this one for now.

# Embeddings

Next, we would use Hugging Face Transformers to load an embedding model and transform the text into a numerical vector. Specifically, we would use sentence embedding to transform the whole sentence.

The recommendation system would be based on the embedding from all the anime “description” we will perform soon. 

We would use the cosine similarity method, which measures the similarity of two vectors. 
By measuring the similarity between the anime “description” embedding and the user's query input embedding, we can get precise items to recommend.

The embedding similarity approach sounds simple, but it can be powerful compared to the classic recommendation system model, as it can capture the semantic relationship between words and provide contextual meaning for the recommendation process.

We would use the embedding model sentence transformers from the Hugging Face for this tutorial. To transform the sentence into embedding, we would use the following code.

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

def get_embeddings(sentences):
  encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

  with torch.no_grad():
      model_output = model(**encoded_input)

  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

  sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

  return sentence_embeddings

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

C:\Users\Deepali\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Deepali\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Try the embedding process and see the vector result with the following code. However, I would not show the output as it’s pretty long.

In [5]:
sentences = ['Some great movie', 'Another funny movie']
result = get_embeddings(sentences)
print("Sentence embeddings:")
# print(result)

Sentence embeddings:


To make things easier, Hugging Face maintains a Python package for embedding sentence transformers, which would minimize the whole transformation process in 3 lines of code. Install the necessary package using the code below.

In [ ]:
pip install -U sentence-transformers

In [10]:
# Then, we can transform the whole anime “description” with the following code.

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

anime_embeddings = model.encode(df['description'].tolist())

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

# With the embedding database is ready, we would create a function to take user input and perform cosine similarity as a recommendation system.

def get_recommendations(query, embeddings, df, top_n=5):
    query_embedding = model.encode([query])
    similarities = cosine_similarity(query_embedding, embeddings)
    top_indices = similarities[0].argsort()[-top_n:][::-1]
    return df.iloc[top_indices]

Now that everything is ready, we can try the recommendation system. Here is an example of acquiring the top five anime recommendations from the user input query.

In [12]:
query = "Funny anime I can watch with friends"
recommendations = get_recommendations(query, anime_embeddings, df)
print(recommendations[['name', 'genre']])

                                          name  \
7363  Sentou Yousei Shoujo Tasukete! Mave-chan   
8140            Anime TV de Hakken! Tamagotchi   
4294      SKET Dance: SD Character Flash Anime   
1061                        Isshuukan Friends.   
2850                       Oshiete! Galko-chan   

                                             genre  
7363  Comedy, Parody, Sci-Fi, Shounen, Super Power  
8140          Comedy, Fantasy, Kids, Slice of Life  
4294                       Comedy, School, Shounen  
1061        Comedy, School, Shounen, Slice of Life  
2850                 Comedy, School, Slice of Life  
